In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neurd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from mesh_tools import trimesh_utils as tu
from python_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

In [4]:
segment_id = 864691135510518224

# Load the appropriate volume data inferface object

In [5]:
from neurd.vdi_microns import volume_data_interface as vdi
#vdi.set_parameters_for_directory_modules()

In [13]:
synapse_filepath = str(Path(f'./{segment_id}_synapses.csv').absolute())
synapse_filepath

'/neurd_packages/NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Double_Soma/864691135510518224_synapses.csv'

In [14]:
from python_tools import pandas_utils as pu
df = pu.csv_to_df(synapse_filepath)
df

,prepost,synapse_id,synapse_x,synapse_y,synapse_z,synapse_size,segment_id
0,presyn,483766883,1488728,621944,1110480,4384,864691135510518224
1,presyn,478550136,1485480,626880,1111240,2068,864691135510518224
2,presyn,480939462,1481320,565104,1111800,11064,864691135510518224
3,presyn,479220146,1482788,601920,1054840,8636,864691135510518224
4,presyn,468794187,1454888,618200,1093960,3516,864691135510518224
...,...,...,...,...,...,...,...
960,postsyn,483712349,1501720,609480,1075880,17880,864691135510518224
961,postsyn,492052366,1524840,622592,1056880,8680,864691135510518224
962,postsyn,486032429,1492720,571480,1029520,2244,864691135510518224
963,postsyn,477362400,1476856,585624,1113440,18312,864691135510518224


In [15]:
vdi.set_synapse_filepath(
    synapse_filepath
)

# Data Prep: Loading mesh

In [17]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = True
)
mesh

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

<trimesh.Trimesh(vertices.shape=(154713, 3), faces.shape=(323535, 3))>

In [ ]:
from python_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 1: Decimation

In [ ]:
"""
NOTE: an already decimated mesh is used for the example
in order to speed up time, that's why the mesh_dec
object will be overwritten with original mesh object
"""

In [ ]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

In [ ]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

# Step 2: Soma Identification

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [ ]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

In [ ]:
# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

# soma_indentification

In [ ]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

In [ ]:
soma_products.soma_run_time

In [ ]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

In [ ]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

In [ ]:
from python_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

In [ ]:
nviz.visualize_neuron_limbs(neuron_obj)

In [ ]:
nviz.visualize_neuron_specific_limb(neuron_obj,"L2")

In [ ]:
ipvu.plot_objects(neuron_obj[2][0].mesh)

# Calculate statistics

In [ ]:
_ = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
ipvu.plot_objects(
    meshes = [
        neuron_list[0].mesh_from_branches,
        neuron_list[1].mesh_from_branches,
    ],
    meshes_colors=["red","blue"]
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
n1 = neuron_list[0]

In [ ]:
vdi

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
# syn_dict = vdi.segment_id_to_synapse_dict(segment_id)
# nviz.plot_objects(
#     neuron_obj.mesh,
#     scatters=[syn_dict["presyn"]["synapse_coordinates"]]
# )

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

In [ ]:
print(neuron_obj_axon.pipeline_products.cell_type_ax_dendr)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

In [ ]:
print(neuron_obj_proof.pipeline_products.auto_proof)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from python_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh_from_branches,
    meshes = [neuron_obj_proof.mesh_from_branches],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)